<a href="https://colab.research.google.com/github/AlinaOtr/data/blob/master/aabb_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Задание 1
Продуктовая команда провела aabb-тест для проверки влияния доработки на сайте на конверсию, необходимо оценить результаты.
Обозначения по группам:
* 0 -- группа А
* 1 -- группа А контрольная
* 2 -- группа Б
* 3 -- группа Б контрольная  

Оценивать можно как группы по отдельности, так и объединенные.

# Получение и предобработка данных

In [1]:
import pandas as pd
import statsmodels.stats.proportion as proportion

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Это ссылка на файл: https://drive.google.com/file/d/1GLmZvZ_WpfdXp4V-z4wdDH-T_x1xDTy4/view?usp=sharing

In [2]:
!gdown --id 1GLmZvZ_WpfdXp4V-z4wdDH-T_x1xDTy4

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1GLmZvZ_WpfdXp4V-z4wdDH-T_x1xDTy4
To: /content/aabb.csv
100% 19.0M/19.0M [00:00<00:00, 136MB/s]


In [112]:
df = pd.read_csv('/content/aabb.csv')

In [113]:
df.head()

,date,clientId,sessionId,experimentVariant,transactionId,transactionRevenue,itemQuantity
0,2021-09-12,9.416909e+08,1630133172418.188lgetf,0,NaN,NaN,NaN
1,2021-09-03,2.123292e+09,1630582225576.6rat3vw,0,NaN,NaN,NaN
2,2021-09-03,1.980968e+09,1630591180040.rrnkdmbs,0,NaN,NaN,NaN
3,2021-09-03,1.598704e+09,1630616613137.lg5wescf,0,NaN,NaN,NaN
4,2021-09-03,3.200845e+08,1630616660855.ynh87hjs,0,NaN,NaN,NaN


In [114]:
df.isna().mean()

date                  0.000000
clientId              0.000000
sessionId             0.000000
experimentVariant     0.000000
transactionId         0.900817
transactionRevenue    0.900817
itemQuantity          0.900817
dtype: float64

Пропуски в последних трёх столбцах (мы видим, что доли пропусков равны между собой) означают отсутствие покупки в данной сессии. Заменю пропущенные значения в столбцах выручки и количества товаров на 0:


In [115]:
df['transactionRevenue'] = df['transactionRevenue'].fillna(0)
df['itemQuantity'] = df['itemQuantity'].fillna(0)

In [116]:
df.duplicated().sum()

0

Полных дубликатов в данных нет

In [117]:
df.groupby('clientId')['experimentVariant'].agg(['count']).query('count > 1')

,count
clientId,
1.657416e+04,2
1.175252e+05,2
1.672262e+05,2
2.561452e+05,2
2.837772e+05,2
...,...
2.147008e+09,2
2.147050e+09,4
2.147090e+09,2


Кажется странным, что есть клиенты, которые оказались в разных группах. Думаю, их не стоит учитывать при оценке теста. Определю тех пользователей, которые оказались только в одной группе:

In [119]:
clients_list = df.groupby('clientId', as_index = False).agg({'experimentVariant': 'count'}).query('experimentVariant == 1')['clientId']

In [128]:
df = df.query('clientId	in @clients_list')

In [130]:
df.nunique()

date                      52
clientId              196569
sessionId             196569
experimentVariant          4
transactionId          17418
transactionRevenue      7659
itemQuantity             122
dtype: int64

В новом датасете на каждого пользователя приходится всего одна сессия. Тогда получается, что пользователи в данной выборке не возвращаются к нам на сайт и с первого раза принимают решение, покупать сразу или не покупать. Конечно, скорее всего в реальной жизни пользователи возвращаются, но в ходе эксперимента такие пользователи почему-то оказывались в разное врем в разных группах, и я их убрала, чтобы не нарушать чистоту эксперимента.

Посмотрю на распределение пользователей - сколько пользователей в каждой из групп:

In [131]:
df['experimentVariant'].value_counts()

0    50228
1    49751
2    48410
3    48180
Name: experimentVariant, dtype: int64

Проведённый А/В тест можно считать сбалансированным.

In [124]:
def define_group(group):  
    if group == 0 or group == 1:
      test_group = "A"
    else:
      test_group = "B"
    return test_group

In [133]:
df['test_group'] = df.experimentVariant.apply(define_group)

In [134]:
def buy_or_not(revenue):
  if revenue == 0:
    buy = 0
  else:
    buy = 1
  return buy

In [135]:
df['buy'] = df.transactionRevenue.apply(buy_or_not)

In [136]:
df.head()

,date,clientId,sessionId,experimentVariant,transactionId,transactionRevenue,itemQuantity,test_group,buy
0,2021-09-12,9.416909e+08,1630133172418.188lgetf,0,NaN,0.0,0.0,A,0
1,2021-09-03,2.123292e+09,1630582225576.6rat3vw,0,NaN,0.0,0.0,A,0
3,2021-09-03,1.598704e+09,1630616613137.lg5wescf,0,NaN,0.0,0.0,A,0
4,2021-09-03,3.200845e+08,1630616660855.ynh87hjs,0,NaN,0.0,0.0,A,0
5,2021-09-03,4.230444e+08,1630616992877.6af6esi,0,NaN,0.0,0.0,A,0


# Оценка результатов

В качестве целевой метрики по условию задачи была выбрана конверсия - отношение числа посетителей сайта, совершивших покупку, к общему числу посетителей сайта.Коверсия является биномиальной метрикой, поэтому для проверки буду использовать Хи-квадрат. 

H0: Конверсии в тестовой и контрольной группе не отличаются  
H1: Конверсия в группе "В" выше, чем в группе "А"  



Значения конверсий в группах "А" и "А контрольной" не должны значимо отличаться. Проверю это:

In [181]:
metrics_a = df.query('test_group == "A"').groupby('experimentVariant', as_index = False).agg({'clientId': 'count', 'buy': 'sum'})
metrics_a['conversion'] = round(metrics_a.buy/metrics_a.clientId *100, 2)
metrics_a

,experimentVariant,clientId,buy,conversion
0,0,50228,4290,8.54
1,1,49751,4454,8.95


In [195]:
new_names_a = ['experimentVariant', 'clients_count', 'buys_count', 'conversion']
metrics_a.set_axis(new_names_a, axis = 'columns', inplace = True)
metrics_a

,experimentVariant,clients_count,buys_count,conversion
0,0,50228,4290,8.54
1,1,49751,4454,8.95


In [196]:
chi2stat_a, pval_a, table_a = proportion.proportions_chisquare(metrics_a['buys_count'], metrics_a['clients_count'])

In [197]:
alpha = 0.05

In [192]:
pval_a < alpha

True

In [198]:
pval_a

0.021278508499492532

Отлично, статистической разницы между группами "А" и "А контрольная" нет - так и должно быть.

Посчитаю значения конверсий в объединённой группе А и объединённой группе В:

In [185]:
metrics_test_group = df.groupby('test_group', as_index = False).agg({'clientId': 'count', 'buy': 'sum'})
metrics_test_group['conversion'] = round(metrics_test_group.buy/metrics_test_group.clientId *100, 2)
metrics_test_group

,test_group,clientId,buy,conversion
0,A,99979,8744,8.75
1,B,96590,8674,8.98


In [186]:
new_names = ['test_group', 'clients_count', 'buys_count', 'conversion']
metrics_test_group.set_axis(new_names, axis = 'columns', inplace = True)
metrics_test_group

,test_group,clients_count,buys_count,conversion
0,A,99979,8744,8.75
1,B,96590,8674,8.98


In [167]:
chi2stat, pval, table = proportion.proportions_chisquare(metrics_test_group['buys_count'], metrics_test_group['clients_count'])

In [173]:
pval < alpha

False

Статистически значимой разницы в конверсии не обнаружено, так как p-value выше выбранного порога ошибки первого рода. Посмотрю, какая была **мощность теста**:

In [205]:
import statsmodels.stats.power as smp
import numpy as np

In [206]:
chipower = smp.GofChisquarePower()

In [216]:
conversion_control = metrics_test_group['conversion'].values[0] / 100
conversion_test = metrics_test_group['conversion'].values[1] / 100

Количество наблюдений в одной из групп (берём меньшее из чисел):

In [212]:
nobs = min(metrics_test_group['clients_count'])
nobs

96590

Effect size вычисляется как корень от квадрата разности конверсий в контрльной и тестовой группе, делённого на конверсию в контрольной группе:

In [214]:
def chi2_effect_size(p0, p1):
  return np.sqrt(((p0-p1)**2 / p0))

In [217]:
chipower.solve_power(effect_size =chi2_effect_size(conversion_control, conversion_test),
                     nobs = nobs,
                     alpha = pval,
                     power = None)

0.9044708163480513

90% - это хороший показатель мощности. Это значит, что с 90% вероятностью мы бы увидели статистически значимое отличие.

**Вывод:** статистически значимой разницы в конверсии не обнаружено, данную доработку не имеет смысла делать.